# Procedural elements

There are cases where you construct a logic out of a chain of primitive elements.
Take a gray counter, for example. It is used often for cross-clock-domain negotations
(dual clock FIFO fill counters).


## Gray counter

This gray counter example is a bit 'overdone' in comparison to the classical
examples to demonstrate configureability and procedural concatenation of elements.

### Gray 'primitive'

This `gray_unit` is a (configureable) element to construct gray counters of arbitrary length.

In [1]:
from myirl.emulation.myhdl import *

_in = Signal
_out = Signal.Output

@block
def gray_unit(
    clk : ClkSignal,
    ce : _in, reset : ResetSignal,
    flavor : _in,
    ia : _in,
    ib: _in,
    oa : _out,
    ob : _out,
    NEW_STYLE = False,
    *, INITVAL = False
):
    a = Signal(INITVAL, name='a')

    @always_seq(clk.posedge, reset)
    def worker():
        if ce == 1:
            a.next = a ^ ((ia ^ flavor) & ib)
            
    if NEW_STYLE:
        wire0 = oa.wireup(a)
        wire1 = ob.wireup(ib & ~(ia ^ flavor)) 
    else: 
        @always_comb
        def assign():
            oa.next = a
            ob.next = ib & ~(ia ^ flavor)

    return instances()

From this element, we construct a gray counter using a few auxiliaries:

In [2]:
import myhdl

def graycode(v, size):
    bv = intbv(v)[size:]
    z = bv ^ myhdl.concat("0", bv[size:1])
    # print "Gray code:", z, len(bv)
    return intbv(z)[size:]

def parity(v):
    """Return parity bit from value v"""
    p = False
    for i in v:
        if i:
            p = not p

    return p

In [3]:
@block
def gray_counter(
    clk : ClkSignal,
    ce : Signal,
    reset : ResetSignal,
    rval : intbv,
    direction : Signal,
    val : Signal.Output
):
        
    n = len(val)
    p = parity(rval)
    
    z = not (p ^ direction) # Important to use `not` to get boolean result
    
    u = [ Signal(bool(z), name="u%d" % (i)) for i in range(n+1) ]
    u0 = u[0]

    v = [ Signal(bool(), name='v%d' % i) for i in range(n+1) ]
    s = Signal(bool())

    tmp = list(reversed(u))[:-1]
    out = concat(*tmp)
    inst_gray = []

    @always_seq(clk.posedge, reset)
    def worker():
        if ce == 1:
            u0.next = ~u0

    wires = [
        val.wireup(out),
        v[0].wireup(True)      
    ]
            
    @always_comb
    def assign():
        if direction == True:
            s.next = u[n-1] ^ u[n]
        else:
            s.next = u[n-1] | u[n]


    for i in range(1, n):
        inst_gray.append(gray_unit(clk, ce, reset, direction, u[i-1], v[i-1], u[i], v[i], INITVAL = rval[i-1]))

    # and MSB:
    inst_gray.append(    gray_unit(clk, ce, reset, direction, s,      v[n-1], u[n], v[n], INITVAL = rval[n-1]))

    return instances()

## Simulation

For fun, we create a `Waveform` class from an Iterator

In [4]:
from myirl.loops import Iterator

class Waveform(Iterator):
    def __init__(self, val, name = 'waveform'):
        it = [ True if i == '1' else False for i in val]
        super().__init__(it, name)
    def convert(self, tgt, tsz = None):
        return "True" if self.val else "False"
    def resolve_type(self):
        return bool

In [5]:
from myirl.test.common_test import gen_osc

@block
def tb_gray(SIZE):
    clk = ClkSignal(name='clk')
    ce = Signal(bool(), name='ce')
    direction = Signal(bool(1))
    direction.init = True

    reset = ResetSignal(0, 1, isasync = False)
    dout = Signal(intbv()[SIZE:], name='dout')

    o = Signal(intbv()[SIZE:])    
    rv = graycode(0, SIZE)
    gc = gray_counter(clk, ce, reset, rv, direction, o, USE_ALIAS=True)
    
    osc = gen_osc(clk, 4)

    @always_comb
    def assign():
        dout.next = o
        
    @instance
    def stim():
        ce.next = False
        direction.next = False
        reset.next = True
        yield delay(10)
        reset.next = False
        for i in Waveform("0011111111100000"):
            ce.next = i       
            yield clk.negedge
        direction.next = True
        for i in Waveform("001111000111110"):
            ce.next = i       
            yield clk.negedge

        raise StopSimulation


    return instances()

from myirl import targets
from myirl.test.common_test import run_ghdl

def run_testbench(SIZE):
    tb = tb_gray(SIZE)
    files = tb.elab(targets.VHDL, elab_all=True)
    run_ghdl(files, tb, debug = True, vcdfile='tb_gray.vcd')
    return files

    
GRAY_COUNTER_SIZE = 4
f = run_testbench(GRAY_COUNTER_SIZE)

Creating process 'gray_counter/worker' with sensitivity (clk'rising, <reset>)
Using default for NEW_STYLE: False
 NEW_STYLE: use default False 
Creating process 'gray_unit/worker' with sensitivity (clk'rising, <reset>)
Using default for NEW_STYLE: False
 NEW_STYLE: use default False 
Using default for NEW_STYLE: False
 NEW_STYLE: use default False 
Using default for NEW_STYLE: False
 NEW_STYLE: use default False 
Creating sequential 'tb_gray/stim' 
 Elaborating component gray_unit_s1_s1_s1_s1_s1_s1_s1_s1_0_0 
 Writing 'gray_unit' to file /tmp/myirl_top_tb_gray_3oyr0l9o/gray_unit.vhdl 
 Elaborating component gray_counter_s1_s1_s1__intbv_s1_s4 
 Writing 'gray_counter' to file /tmp/myirl_top_tb_gray_3oyr0l9o/gray_counter.vhdl 
 Elaborating component tb_gray_4 
 Writing 'tb_gray' to file /tmp/myirl_top_tb_gray_3oyr0l9o/tb_gray.vhdl 
 Creating library file /tmp/myirl_module_defs_t_8wxf46/module_defs.vhdl 
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/t

In the non-parametrized variant, two implementations of `gray_unit` are emitted. We dump the first:

In [6]:
! cat -n {f[0]}

     1	-- File generated from source:
     2	--     /tmp/ipykernel_42709/2642985506.py
     3	-- (c) 2016-2021 section5.ch
     4	-- Modifications may be lost, edit the source file instead.
     5	
     6	library IEEE;
     7	use IEEE.std_logic_1164.all;
     8	use IEEE.numeric_std.all;
     9	
    10	library work;
    11	
    12	use work.txt_util.all;
    13	use work.myirl_conversion.all;
    14	
    15	entity gray_unit is
    16	    generic (
    17	        INITVAL: boolean := FALSE
    18	    );
    19	    port (
    20	        clk : in std_ulogic;
    21	        ce : in std_ulogic;
    22	        reset : in std_ulogic;
    23	        flavor : in std_ulogic;
    24	        ia : in std_ulogic;
    25	        ib : in std_ulogic;
    26	        oa : out std_ulogic;
    27	        ob : out std_ulogic
    28	    );
    29	end entity gray_unit;
    30	
    31	architecture MyIRL of gray_unit is
    32	    -- Local type declarations
    33	    -- Signal declarations
    34	    signal a : st

## Waveform display

In [7]:
import wavedraw
import nbwavedrom

TB = "tb_gray"

# Configure waveform:
cfg = wavedraw.config(TB, {
    '.clk' : None,
    '.reset' : None, '.dout[%d:0]' % (GRAY_COUNTER_SIZE-1) : None, '.ce' : None})

cfg[TB + '.inst_gray_counter_0.reset'] = None

u = [ TB + ".inst_gray_counter_0.u%d" % i for i in range(GRAY_COUNTER_SIZE+1) ]

for s in u:
    cfg[s] = None

waveform = wavedraw.vcd2wave("tb_gray.vcd", TB + '.clk', cfg, delta = 10)
    
nbwavedrom.draw(waveform)

tb_gray.clk
tb_gray.reset
tb_gray.dout[3:0]
tb_gray.ce
tb_gray.inst_gray_counter_0.reset
tb_gray.inst_gray_counter_0.u0
tb_gray.inst_gray_counter_0.u1
tb_gray.inst_gray_counter_0.u2
tb_gray.inst_gray_counter_0.u3
tb_gray.inst_gray_counter_0.u4


### Observations

* The gray counter begins with a start value (`rval`) corresponding to an index
* Whenever `ce` is active, it advances according to the `direction` parameter
* Note that on every advance, only one bit flips in the counter output (`u[1..n]` bits)

### Exercises

* Change direction
* Try another start value

## Alternative implementation via yosys

For a more realistic, more compact implementation, we use only a bit of procedural connection generator syntax.
We also skip direction (downcounting) and initializer support.

In [8]:
@block
def gray_counter1(clk : ClkSignal,
                  enable : Signal,
                  reset : ResetSignal,
                  gray_count :Signal.Output):

    n = len(gray_count)
    gray_bits = [ Signal(bool(), name="u%d" % i) for i in range(n) ]
    code, work = [ Signal(modbv()[n:]) for _ in range(2) ]
    flags = [ Signal(bool(), name="flags%d" % i) for i in range(n + 1) ]
    toggle = Signal(bool(1))

    # This creates connection instances:
    connections = [
        flags[0].wireup(False),
        code.wireup(concat(*reversed(gray_bits))),
        work.wireup(concat("1", code[n-2:], toggle))
    ]  
        
    for i in range(n):
        v = work[i] & ~flags[i]
        connections += [
            gray_bits[i].wireup(v ^ gray_count[i]),
            flags[i + 1].wireup(flags[i] | v )
        ]
                
    @always_seq(clk.posedge, reset)
    def worker():
        if enable == True:
            gray_count.next = code
            toggle.next = ~toggle

    return instances()

### CXXRTL simulation

This time we create a test bench based on yosys' CXXRTL simulator backend.
The above code is elaborated implicitely using direct RTL transfer, compiled into C++ code and executed. The result of this is a trace dump into a VCD file.

In [9]:
from myirl.test.common_test import gen_osc

from yosys.simulator import CXXRTL

from simulation import *

@sim.testbench(CXXRTL, time_unit = 'ns')
def tb_gray1(SIZE):
    clk = ClkSignal(name='clk')
    ce = Signal(bool(), name='ce')

    reset = ResetSignal(0, 1, isasync = True)
    dout = Signal(intbv()[SIZE:], name='dout')

    dout = Signal(intbv()[SIZE:])    
    gc = gray_counter1(clk, ce, reset, dout)
    
    @always(delay(2))
    def clkgen():
        clk.next = ~clk
    
    @instance
    def stim():
        ce.next = False
        reset.next = True
        yield delay(6)
        reset.next = False
        for i in Waveform("0011111111111111111111111111111111000"):
            ce.next = i       
            yield clk.negedge

        raise StopSimulation

    return instances()    

Run the test bench for a few cycles.

In [10]:
GRAY_COUNTER_SIZE = 5
tb = tb_gray1(GRAY_COUNTER_SIZE)
tb.run(180)

Creating process 'gray_counter1/worker' with sensitivity (clk'rising, <reset>)
 Adding module with name `gray_counter1` 
 FINALIZE implementation `gray_counter1` of `gray_counter1` 
Compiling /tmp/myirl_top_gray_counter1_h97ejyp2/gray_counter1_1516.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_top_gray_counter1_h97ejyp2/gray_counter1_1516.pyx
running build_ext
building 'gray_counter1_1516' extension
creating build/temp.linux-x86_64-3.9/tmp/myirl_top_gray_counter1_h97ejyp2
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -ffile-prefix-map=/build/python3.9-RNBry6/python3.9-3.9.2=. -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -ffile-prefix-map=/build/python3.9-RNBry6/python3.9-3.9.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/../ -I/tmp/myirl_top_gray_counter1_

### Waveform display

The resulting waveform is displayed using a signal filter configuration:

In [11]:
TB = ""

# Configure waveform:
cfg = wavedraw.config(TB, 
      { '.reset' : None, '.clk' : None, '.code': None, '.enable' : None }
)

u = [ TB + ".u%d" % i for i in range(GRAY_COUNTER_SIZE-1) ]

for s in u:
    cfg[s] = None

waveform = wavedraw.vcd2wave("tb_gray1.vcd", '.clk', cfg, delta = 1)
    
nbwavedrom.draw(waveform)

.reset
.clk
.code
.enable
.u0
.u1
.u2
.u3
